In [2]:
import dataset 
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import time
import pickle
from pathlib import Path
import math

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from  numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
print(tf.__version__)
tf.get_default_graph()

1.8.0


load du lieu

In [4]:
# khoi tao gia tri load du lieu




train_path= r"D:\Hoang\data_kata_khoaluan"

classes = dataset.get_labels(train_path)

print(classes)

num_classes = len(classes)
print(num_classes)

['a_k', 'chi_k', 'e_k', 'fu_k', 'ha_k', 'he_k', 'hi_k', 'ho_k', 'i_k', 'ka_k', 'ke_k', 'ki_k', 'ko_k', 'ku_k', 'ma_k', 'me_k', 'mi_k', 'mo_k', 'mu_k', 'na_k', 'ne_k', 'ni_k', 'no_k', 'nu_k', 'n_k', 'o_k', 'ra_k', 're_k', 'ri_k', 'ro_k', 'ru_k', 'sa_k', 'se_k', 'shi_k', 'so_k', 'su_k', 'ta_k', 'te_k', 'to_k', 'tsu_k', 'u_k', 'wa_k', 'wo_k', 'ya_k', 'yo_k', 'yu_k']
46


In [5]:
#load du lieu 
        
validation_size = 0.1
test_size = 0.2
img_size = 64
num_input_channels = 1

class DataCompression(object):
    def __init__(self,data):
        self.train = data.train
        self.valid = data.valid
        self.test = data.test
        
p = Path.cwd()
files=[]
for file in p.iterdir():
    x=file.parts[-1]
    files.append(x)
    
if "train_data.file" not in files:
    print("Get and Compress data")
    data_train=dataset.read_train_sets(train_path=train_path,image_size=img_size,test_size=test_size,validation_size=validation_size)
    data_train = DataCompression(data_train)
    p=str(p.joinpath("train_data.file"))
#     print(p)
    with open(p, "wb") as f:
        pickle.dump(data_train, f, pickle.HIGHEST_PROTOCOL)
    
#read data from file data compression
p=Path.cwd()
p=str(p.joinpath("train_data.file"))
print(p)
with open(p,"rb") as f:
    print("uncompress data")
    data = pickle.load(f)
    


C:\Users\buico\b_khoaluan\train_data.file
uncompress data


In [6]:
print(data.train.images.shape)
print(data.valid.images.shape)
print(data.test.images.shape)

(545144, 64, 64, 1)
(77877, 64, 64, 1)
(155755, 64, 64, 1)


# Xây dựng hàm cho CNN

Khởi tạo giá trị

In [7]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_input_channels], name='x')

    # labels
    y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

    keep_prob=tf.placeholder(tf.float32,name="keep_prob")
    
    y_true_cls = tf.argmax(y_true, axis=1)

In [8]:
# cau truc CNN M6-1

filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64

filter_size_conv4 = 3
num_filters_conv4 = 64
    
fc_layer_size = 256

In [9]:
def create_weights(shape,name):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05),name=name)

def create_biases(size,name):
    return tf.Variable(tf.constant(0.05, shape=[size]),name=name)

In [10]:
def create_conv_layer(input,
               num_input_channels, # so channels của từng ảnh
               filter_size,    # kích thước filter, kenel, 
               num_filters,  # số lượng đầu ra cho layer sau
               use_maxpool=False, #sử dụng max_pooling k?
               name = "conv_x" ):  
    with tf.name_scope(name):
        #tao weight và bias cho lop convolutional nay/
#         print("input {}: {}".format(name,input))

        name_weights = name+"_weights"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[filter_size, filter_size, num_input_channels, num_filters],name=name_weights)
#         print("weights {}: {}".format(name,weights))

        name_biases = name+"_biases"
        with tf.name_scope("biases"):
            biases = create_biases(num_filters,name=name_biases)
        ## xay dung convolution layer, layer trả về có dimension nhu input
#         with tf.device('/gpu:0'):
        layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
#         print("layer {}: {}".format(name,layer))

        layer += biases
        layer = tf.nn.relu(layer)

        if use_maxpool:
            layer = tf.nn.max_pool(value=layer,
                                    ksize=[1, 2, 2, 1],
                                    strides=[1, 2, 2, 1],
                                    padding='SAME')


#         store in histogram
        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        
        layer = tf.identity(layer,name=name)
        return layer

def create_flatten_layer(layer, name="flatten"):
    with tf.name_scope(name):
        # chuyen mỗi ảnh thanh mot vector, moi anh sẽ có kích thước phẳng là: img_size * img_size *num_channels

        #dimension layer: batch_size,img_size,img_size,num_channels 

        #lấy số lượng data
        layer_shape = layer.get_shape()

        # số lượng feature mỗi ảnh = img_size * img_size *num_channels
        num_features = layer_shape[1:4].num_elements()

        # reshape lại data ví dụ (10,2,2,5) // có 10 ảnh và mỗi ảnh có 2*2*5=20 feature => flat_layer: (10,20) 
        layer = tf.reshape(layer, [-1, num_features])
#         print("layer {}: {}".format(name,layer))
        layer = tf.identity(layer,name=name) # set name for layer
        return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=False, 
             use_dropout=False,
             name = "fc"):
    
    with tf.name_scope(name):
        name_weights = name+"_weights"
        name_biases = name+"_biases"
        with tf.name_scope("weights"):
            weights = create_weights(shape=[num_inputs, num_outputs],name=name_weights)
        with tf.name_scope("biases"):
            biases = create_biases(num_outputs,name=name_biases)
    
#         with tf.device('/gpu:0'):
        layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer)
        if use_dropout:
             layer=tf.nn.dropout(layer,keep_prob);
#         print("layer {}: {}".format(name,layer))

        tf.summary.histogram("weights",weights)
        tf.summary.histogram("biases",biases)
        tf.summary.histogram("layer",layer)
        
        layer = tf.identity(layer,name=name)
        return layer

# Xây dựng cấu trúc CNN
conv3-32 -> conv3-32 ->maxpool -> conv3-64 -> conv3-64 -> maxpool ->fc-256

In [11]:
conv1 =  create_conv_layer(input=x,num_input_channels=num_input_channels,
                           filter_size=filter_size_conv1,num_filters=num_filters_conv1,use_maxpool=False, name="conv1")

conv2 = create_conv_layer(input=conv1,num_input_channels=num_filters_conv1,
                          filter_size=filter_size_conv2,num_filters=num_filters_conv2,use_maxpool=True, name="conv2")

conv3 = create_conv_layer(input=conv2,num_input_channels=num_filters_conv2,
                          filter_size=filter_size_conv3,num_filters=num_filters_conv3,use_maxpool=False, name="conv3")

conv4 = create_conv_layer(input=conv3,num_input_channels=num_filters_conv3,
                          filter_size=filter_size_conv4,num_filters=num_filters_conv4,use_maxpool=True, name="conv4")

layer_flat = create_flatten_layer(conv4)

num_feature = layer_flat.get_shape()[1:2].num_elements()

fc1 = create_fc_layer(input=layer_flat,num_inputs=num_feature,num_outputs=fc_layer_size,use_relu=True,use_dropout=True,name="fc1")


fc2 = create_fc_layer(input=fc1,num_inputs=fc_layer_size,num_outputs=num_classes,use_relu=False,use_dropout=True,name="fc2")



predict

In [12]:
with tf.name_scope("Train"):
#     with tf.device('/gpu:0'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2,
                                                        labels=y_true)
    with tf.name_scope("loss"):
        loss = tf.reduce_mean(cross_entropy)
        tf.summary.scalar("loss",loss)
    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)


with tf.name_scope("Test"):
    y_pred = tf.nn.softmax(fc2,name='y_pred')
    y_pred_cls = tf.argmax(y_pred, axis=1)
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    
    tf.summary.scalar("accuracy",accuracy)



In [13]:
my_config =tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)
my_config.gpu_options.allow_growth = True
session = tf.Session(config=my_config)

session.run(tf.global_variables_initializer())

saver = tf.train.Saver()

merged =tf.summary.merge_all();
train_writer= tf.summary.FileWriter(r".\model_graph\graph\M6\train",session.graph)
valid_writer=tf.summary.FileWriter(r".\model_graph\graph\M6\valid",session.graph)

In [ ]:
def train(num_epochs, batch_size):
    num_iteration = int(data.train.images.shape[0]/batch_size)

    print_loss_iteration = int(num_iteration /5);
    total_iterations =0
    for epoch in range(num_epochs):
        for i in range(num_iteration):

            x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
    #         print(y_true_batch.shape)
            feed_dict_train = {x: x_batch, y_true: y_true_batch, keep_prob: 0.5}    

            session.run(train_step, feed_dict=feed_dict_train )
            
            if (i % print_loss_iteration == 0 or i==num_iteration-1) and i>0: 
                x_valid_batch, y_valid_batch, iname_valid_batch,cls_valid_batch = data.train.next_batch(500)
                
                feed_dict_valid = {x: x_valid_batch, y_true: y_valid_batch, keep_prob:1}
                feed_dict_train = {x: x_batch, y_true: y_true_batch, keep_prob: 1}

                valid_loss,valid_accuracy = session.run([loss,accuracy], feed_dict=feed_dict_valid)
 
                train_loss,train_accuracy = session.run([loss,accuracy], feed_dict=feed_dict_train)
                
                total_iterations=total_iterations+print_loss_iteration

                
                print(f"time: {time.ctime()} -- epoch: {epoch} -- iteration: {total_iterations} -- train_loss: {train_loss:.2f} -- train_acc {train_accuracy:.2%} --valid_loss:{valid_loss:.2f} -- valid_acc {valid_accuracy:.2%}")
                if i==num_iteration-1:
                    print(f"save graph -- epoch: {epoch}-- iteration: {i}")
                    train_merged=session.run(merged, feed_dict=feed_dict_train)
                    valid_merged = session.run(merged, feed_dict=feed_dict_valid)
                    train_writer.add_summary(train_merged,epoch)
                    valid_writer.add_summary(valid_merged,epoch)
            
            
        save_path=saver.save(session, r'.\model_graph\model\modelTest.ckpt',global_step=epoch)

def accuracy_test():
    
    num_interations = data.test.images[0]/500
    
    for iteration in range(num_interations):
        x_batch_test, y_true_batch_test, _, cls_batch_test = data.test.next_batch(500)
        feed_dict_test = {x:x_batch,y_true:y_batch,keep_prob:1.0}
        acc=session.run(accuracy,feed_dict =feed_dict_test)
        print(f"iter: {iteration} -- accuracy test : {acc}")

In [ ]:
start_time = time.time()
print(f"start time: {time.ctime(start_time)}")
train(num_epochs=4, batch_size =100)
end_time= time.time()
print(f"end time: {time.ctime(start_time)}")
run_time =(end_time-start_time)/60
print(f"run time: {run_time} m ")

start time: Sun May 27 16:13:28 2018
epoch: 0 -- iteration: 1090 -- train_loss: 3.83 -- train_acc 1.00% --valid_loss:3.83 -- valid_acc 2.40%


In [ ]:
accuracy_test()